In [4]:
import numpy as np
from scipy.integrate import solve_ivp
import pandas as pd

# Constants
F = 8.0
N = 40
dt = 0.05
days = 730
day_steps = int(0.20 / dt)  # Steps per day
sample = int(days / 2 * day_steps)  # Samples for the last year in a two-year period

# Define the Lorenz 96 model function
def L96(t, x, F):
    dxdt = np.zeros(N)
    for i in range(N):
        dxdt[i] = (x[(i + 1) % N] - x[i - 2]) * x[i - 1] - x[i] + F
    return dxdt

# Initial conditions for spin-up phase
X_minusL = np.random.normal(F / 4.0, F / 2.0, N)
L = 360
t_initialspan = (0.0, L)
t_initialeval = np.linspace(t_initialspan[0], t_initialspan[1], int(L / dt + 1))
initialsol = solve_ivp(L96, t_initialspan, X_minusL, args=(F,), method='RK45', t_eval=t_initialeval)
X0 = initialsol.y[:, -1]

# Main simulation over the period of interest
tspan = (0.0, days)
t_eval = np.linspace(tspan[0], tspan[1], int(tspan[1] / dt + 1))
sol = solve_ivp(L96, tspan, X0, args=(F,), method='RK45', t_eval=t_eval)

# Collect data from the second year
X = sol.y[:, int(len(t_eval) / 2):].reshape(N, -1)[:, ::day_steps][:, :sample]

# Create observational data
rng = np.random.default_rng()
normal_random_numbers = rng.normal(loc=0, scale=1, size=X.shape)
X_obs = X + normal_random_numbers

# Export results to CSV
df_rnd = pd.DataFrame(normal_random_numbers)
df_true = pd.DataFrame(X)
df_obs = pd.DataFrame(X_obs)

df_rnd.to_csv('random_noise.csv')
df_true.to_csv('true_data.csv')
df_obs.to_csv('observed_data.csv')
